# Linear Regression - Proje: Ev Fiyatı Tahmini

Bu proje, ev özelliklerine göre fiyat tahmini yapan bir model geliştirir.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.datasets import make_regression

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


## 1. Veri Hazırlama


In [ ]:
# Örnek veri seti (gerçek projede kaggle house prices kullanın)
X, y = make_regression(n_samples=2000, n_features=15, n_informative=12, 
                       n_redundant=3, noise=25, random_state=42)

feature_names = ['Size', 'Bedrooms', 'Bathrooms', 'Age', 'Distance', 
                 'School_Rating', 'Crime_Rate', 'Parking', 'Garden', 
                 'Balcony', 'Elevator', 'Pool', 'Feature_13', 'Feature_14', 'Feature_15']
df = pd.DataFrame(X, columns=feature_names)
df['Price'] = y

print(f"Veri seti boyutu: {df.shape}")
print(f"\nİlk 5 satır:")
print(df.head())
print(f"\nTemel istatistikler:")
print(df.describe())


## 2. Model Eğitimi ve Karşılaştırma


In [ ]:
X = df.drop('Price', axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Farklı modeller
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (alpha=1.0)': Ridge(alpha=1.0),
    'Lasso (alpha=1.0)': Lasso(alpha=1.0),
    'Elastic Net (alpha=1.0)': ElasticNet(alpha=1.0, l1_ratio=0.5)
}

results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    
    results[name] = {
        'R²': r2_score(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'CV R² Mean': cv_scores.mean(),
        'CV R² Std': cv_scores.std()
    }

results_df = pd.DataFrame(results).T
print("Model Karşılaştırması:")
print(results_df.round(4))


## 3. Görselleştirmeler


In [ ]:
# En iyi modeli seç
best_model_name = results_df['R²'].idxmax()
best_model = models[best_model_name]
best_model.fit(X_train_scaled, y_train)
y_pred_best = best_model.predict(X_test_scaled)

# Gerçek vs Tahmin scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_best, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
plt.xlabel('Gerçek Fiyat')
plt.ylabel('Tahmin Edilen Fiyat')
plt.title(f'{best_model_name} - Gerçek vs Tahmin')
plt.grid(True, alpha=0.3)
plt.show()

# Residual plot
residuals = y_test - y_pred_best
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_best, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Tahmin Edilen Fiyat')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)
plt.show()


## 4. Feature Importance (Linear Regression Katsayıları)


In [ ]:
# Linear Regression katsayıları
if hasattr(best_model, 'coef_'):
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Coefficient': best_model.coef_,
        'Abs_Coefficient': np.abs(best_model.coef_)
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print("En Önemli Feature'lar (Katsayı Büyüklüğüne Göre):")
    print(feature_importance.head(10))
    
    # Görselleştir
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance.head(10), x='Abs_Coefficient', y='Feature')
    plt.title('Top 10 Feature Importance')
    plt.xlabel('Absolute Coefficient Value')
    plt.show()
